In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

link = 'https://www.cricbuzz.com/cricket-team'
res = requests.get(link)
soup = BeautifulSoup(res.content, 'html.parser')

sp = soup.find('div',class_ = 'cb-col cb-col-67 cb-nws-lft-col')

### 1. Scraper Team Links

In [2]:
data = []

for teams in sp.find_all('div', class_ = 'cb-col cb-col-50 cb-team-item cb-lst-itm cb-team-lft-item'):
    team_name = teams.find('h2').text.strip()
    team_link = 'https://www.cricbuzz.com' + teams.find('h2').find('a').get('href') + '/players'
    data.append([team_name,team_link])
    
for teams in sp.find_all('div', class_ = 'cb-col cb-col-50 cb-team-item cb-lst-itm cb-team-rght-item'):
    team_name = teams.find('h2').text.strip()
    team_link = 'https://www.cricbuzz.com' + teams.find('h2').find('a').get('href') + '/players'
    data.append([team_name,team_link])

### 2. Scrape Player Links

In [3]:
player_data = []

for team_name, team_link in tqdm(data):
    
    soup = BeautifulSoup(requests.get(team_link).content, 'html.parser')
    sp = soup.find('div', class_ = 'cb-col-67 cb-col cb-left cb-top-zero')
    
    for player in sp.find_all('a', class_ = 'cb-col cb-col-50'):
        player_name = player.text.strip()
        player_link = 'https://www.cricbuzz.com' + player.get('href')
        
        player_data.append([player_name,team_name,player_link])

100%|███████████████████████████████████████████| 37/37 [00:18<00:00,  1.96it/s]


### 3. Scraper Players Data

In [4]:
data = []

for player_name,team_name,player_link in tqdm(player_data):
    
    basic_info = []
    
    try:
        soup = BeautifulSoup(requests.get(player_link).content, 'html.parser')
        sp = soup.find('div',class_ = 'cb-col cb-col-33 text-black')
        player_height = sp.find('div', class_ = 'cb-col cb-col-60').text.strip()
        basic_info.append(player_name)
        basic_info.append(team_name)
        basic_info.append(player_height)
        for i in [player.text.strip() for player in sp.find_all('div', class_ = 'cb-col cb-col-60 cb-lst-itm-sm')][0:3]:
            basic_info.append(i)
        # Scraping Teams
        basic_info.append([player.text.strip() for player in sp.find_all('div', class_ = 'cb-col cb-col-60 cb-lst-itm-sm')][-1]) 

        basic_info += [i.text.strip() for i in soup.find_all('div', class_ = 'cb-col cb-col-25 cb-plyr-rank text-right')]
        
        data.append(basic_info)
        
    except:
        print(player_name, player_link)
        
        pass

 29%|████████████                             | 133/453 [00:48<01:58,  2.69it/s]

Loa Nou https://www.cricbuzz.com/profiles/10425/loa-nou


 30%|████████████▍                            | 137/453 [00:49<01:47,  2.94it/s]

Arun Poulose https://www.cricbuzz.com/profiles/10454/arun-poulose


 37%|███████████████▎                         | 169/453 [01:00<01:31,  3.11it/s]

Zeeshan Siddiqi https://www.cricbuzz.com/profiles/8027/zeeshan-siddiqi


 85%|██████████████████████████████████▉      | 386/453 [02:59<11:29, 10.30s/it]

Mohammad Naveed https://www.cricbuzz.com/profiles/9832/mohammad-naveed


100%|█████████████████████████████████████████| 453/453 [03:28<00:00,  2.17it/s]


### 4. Saving the CSV

In [12]:
df = pd.DataFrame(data, columns = ['name','team','height','date_of_birth','birth_place','role','teams',
                                   'bat_test','bat_odi','bat_t20','bow_test','bow_odi','bow_t20'])

df.to_csv('players.csv', index = False)